In [1]:
using LowLevelFEM, LinearAlgebra
gmsh.initialize()

gmsh.open("nyilt-korgyuru.geo")

[ Info: Precompiling LowLevelFEM [6171b9fb-adbf-4751-adb9-5faded75de07] (cache misses: include_dependency fsize change (4), wrong dep version loaded (2))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up
┌ Warning: Number of threads(1) ≠ logical threads in CPU(4).
└ @ LowLevelFEM ~/Dokumentumok/GitHub/LowLevelFEM.jl/src/general.jl:104


Info    : Reading 'nyilt-korgyuru.geo'...
Info    : Meshing 1D...                                                                                                                   
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Circle)
Info    : [ 10%] Meshing curve 4 (Circle)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 20%] Meshing curve 7 (Line)
Info    : [ 20%] Meshing curve 8 (Line)
Info    : [ 20%] Meshing curve 9 (Circle)
Info    : [ 20%] Meshing curve 10 (Circle)
Info    : [ 20%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Line)
Info    : [ 30%] Meshing curve 13 (Line)
Info    : [ 30%] Meshing curve 14 (Line)
Info    : [ 30%] Meshing curve 15 (Circle)
Info    : [ 30%] Meshing curve 16 (Line)
Info    : [ 30%] Meshing curve 17 (Line)
Info    : [ 30%] Meshing curve 18 (Line)
Info    : [ 40%] Meshing curve 19 (Circle)
Info    : [ 40%] Meshing curve 20 (Li

In [2]:
mat = material("body")
prob = Problem([mat])

Problem("nyilt-korgyuru", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 14508)

In [3]:
n1x(x, y, z) = x
n1y(x, y, z) = y
n2x(x, y, z) = -y
n2y(x, y, z) = x
cs = CoordinateSystem([n1x, n1y, 0], [n2x, n2y, 0])
Q = rotateNodes(prob, "body", cs)

LowLevelFEM.Transformation(sparse([1, 2, 1, 2, 3, 4, 5, 4, 5, 6  …  43519, 43520, 43519, 43520, 43521, 43522, 43523, 43522, 43523, 43524], [1, 1, 2, 2, 3, 4, 4, 5, 5, 6  …  43519, 43519, 43520, 43520, 43521, 43522, 43522, 43523, 43523, 43524], [1.0, -1.971496236724573e-16, 1.971496236724573e-16, 1.0, 1.0, 1.0, -1.971496236724573e-16, 1.971496236724573e-16, 1.0, 1.0  …  0.9976687691905392, -0.06824241336467156, 0.06824241336467156, 0.9976687691905392, 1.0, 0.9976687691905392, -0.06824241336467166, 0.06824241336467164, 0.9976687691905392, 1.0], 43524, 43524), 14508, 3)

In [4]:
M0 = integrate(prob, "right", (x, y, z) -> (x^2 + y^2))

173777.61418690695

In [5]:
supp1 = displacementConstraint("left", uy=0)

fy(x, y, z) = √(x^2 + y^2) / M0 * 10000
load1 = load("right", fy=fy)

elsupp1 = elasticSupport("left", kz=1)
elsupp2 = elasticSupport("right", kx=1)

("right", 1, 0, 0)

In [6]:
f = loadVector(prob, [load1]);

In [7]:
showDoFResults(Q * f, name="load")

1

In [8]:
K = stiffnessMatrix(prob)

43524×43524 SparseArrays.SparseMatrixCSC{Float64, Int64} with 2865758 stored entries:
⎡⣿⣿⣿⠹⠗⡗⢿⠾⠽⠷⡷⣿⣺⣋⣓⣗⣧⣽⢨⡅⠅⠅⠛⠛⠻⠶⠾⠿⠿⣷⣶⡇⠀⡅⠀⣙⣛⣻⣶⣾⎤
⎢⣟⡛⢻⣶⣀⡇⠀⢈⣐⣁⡀⠙⠈⠀⠁⠁⠁⠉⠙⠛⠛⠛⢳⣤⣀⠀⢀⣀⣀⠀⠀⠛⠛⠛⠛⠋⠀⠉⠀⠈⎥
⎢⢽⠥⠤⠼⠿⣧⣠⢠⠈⠈⠁⠀⠀⠤⠤⠄⠀⠀⠀⠀⠀⠀⠬⠿⢿⣀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠠⠤⠤⠀⠀⎥
⎢⣻⡗⡀⢀⠀⣚⣻⣾⣀⠀⡒⠒⠒⠀⠀⣀⣀⣀⠀⠀⠀⠀⠀⠀⢐⣾⣿⡀⠐⠒⠒⠂⠀⠀⠀⠀⠀⢀⣀⣀⎥
⎢⢷⡇⠔⢸⡂⠀⠀⠘⠿⣧⡆⠀⠀⠀⠀⠀⠀⠀⠠⠀⠆⠆⠀⠀⠀⠀⠸⢿⣷⠀⠀⠆⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣽⣯⣄⠈⠁⠀⢸⠈⠈⠉⣿⣿⣠⠀⠀⠀⠀⢠⢠⡄⠀⠀⠀⠀⠀⠀⠈⠀⢈⣷⣦⡄⠀⡄⠀⠀⠀⠀⠀⠀⎥
⎢⡾⢺⠂⠀⠀⡄⠘⠀⠀⠀⠀⠚⢻⣶⡒⡖⠒⠒⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠂⠀⠀⠀⠶⣶⣒⠒⠒⎥
⎢⢽⢼⠅⠀⠀⠇⠀⢠⠀⠀⠀⠀⢸⠬⠿⣧⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠭⠓⢯⣄⡀⎥
⎢⣍⣿⡅⠀⠀⠀⠀⢸⠀⠀⠀⣀⢸⠀⠀⠉⡻⣮⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⡀⠀⠀⠘⠺⢭⎥
⎢⠆⠶⣷⠀⠀⠀⠀⠀⠀⠂⠀⠶⠀⠀⠀⠀⠀⠘⢻⣶⡖⠒⠂⠀⠀⠀⠀⠀⠀⠀⠀⠒⠒⠾⣿⡇⠀⠀⠀⠀⎥
⎢⠅⠅⣿⠀⠀⠀⠀⠀⠨⠅⠀⠀⠀⠀⠀⠀⠀⠀⢸⠉⠿⣧⡄⠀⠀⠀⠀⠀⠀⠀⠀⣽⡶⠄⠀⠁⠀⠀⠀⠀⎥
⎢⣿⠀⠙⣶⣦⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠉⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠉⠁⠀⠀⠀⠀⠀⠀⎥
⎢⢻⡆⠀⠘⠛⢳⣰⣴⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣾⡇⠀⢰⠀⠀⠛⠻⣶⣆⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢿⣧⠀⠘⠂⠀⢰⠀⠙⠛⢦⣴⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠼⠿⣤⠀⠀⠀⠸⠀⠠⠄⠈⠿⠳⠀⠀⠀⠀⠀⢠⠀⣄⣤⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠄⠤⣿⠀⠀⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠀⠀⢠⣸⡄⠘⠏⠇⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⎥
⎢⣄⢠⡿⠀⠀⡀⠀⠀⠀⠀⠀⠀⢠⡄⡄⡄⠀⠈⠿⠿⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⎥
⎢⣿⣸⡄⠀⠀⡇⠀⢀⠀⠀⠀⠀⢸⢻⡽⣄⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⎥
⎣⣸⣿⡀⠀⠀⠀⠀⢸⠀⠀⠀⠀⢸⠀⠀⠹⡞⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⎦

In [9]:
C1 = elasticSupportMatrix(prob, [elsupp1])

43524×43524 SparseArrays.SparseMatrixCSC{Float64, Int64} with 2770 stored entries:
⎡⣿⢟⡫⠀⠀⠃⠀⠀⠀⠀⠇⠀⠀⠀⠀⡃⠀⠀⠀⠀⠀⠅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠋⠊⠑⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠤⠀⠀⠀⠀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠉⠁⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠤⠠⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠄⠄⠠⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎦

In [10]:
C2 = elasticSupportMatrix(prob, [elsupp2])
C2 = Q * C2 * Q'

43524×43524 SparseArrays.SparseMatrixCSC{Float64, Int64} with 11012 stored entries:
⎡⣿⢟⣍⠀⠀⠀⠀⠸⠀⠀⠀⠀⢘⠀⠀⠀⠀⢨⢈⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠃⠙⠛⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠘⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣀⡀⠀⠀⠀⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠒⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠆⠐⠶⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠰⠆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎦

In [12]:
K1, f1 = applyBoundaryConditions(Q' * (K + C1 + C2) * Q, f, [supp1])

43524×43524 SparseArrays.SparseMatrixCSC{Float64, Int64} with 2833629 stored entries:
⎡⣿⣿⣿⠹⠗⡗⢿⠾⠽⠷⡷⣿⣺⣋⣓⣗⣧⣽⢨⡅⠅⠅⠛⠛⠻⠶⠾⠿⠿⣷⣶⡇⠀⡅⠀⣙⣛⣻⣶⣾⎤
⎢⣟⡛⢻⣶⣀⡇⠀⢈⣐⣁⡀⠙⠈⠀⠁⠁⠁⠉⠙⠛⠛⠛⢳⣤⣀⠀⢀⣀⣀⠀⠀⠛⠛⠛⠛⠋⠀⠉⠀⠈⎥
⎢⢽⠥⠤⠼⠿⣧⣠⢠⠈⠈⠁⠀⠀⠤⠤⠄⠀⠀⠀⠀⠀⠀⠬⠿⢿⣀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠠⠤⠤⠀⠀⎥
⎢⣻⡗⡀⢀⠀⣚⣻⣾⣀⠀⡒⠒⠒⠀⠀⣀⣀⣀⠀⠀⠀⠀⠀⠀⢐⣾⣿⡀⠐⠒⠒⠂⠀⠀⠀⠀⠀⢀⣀⣀⎥
⎢⢷⡇⠔⢸⡂⠀⠀⠘⠿⣧⡆⠀⠀⠀⠀⠀⠀⠀⠠⠀⠆⠆⠀⠀⠀⠀⠸⢿⣷⠀⠀⠆⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣽⣯⣄⠈⠁⠀⢸⠈⠈⠉⣿⣿⣠⠀⠀⠀⠀⢠⢠⡄⠀⠀⠀⠀⠀⠀⠈⠀⢈⣷⣦⡄⠀⡄⠀⠀⠀⠀⠀⠀⎥
⎢⡾⢺⠂⠀⠀⡄⠘⠀⠀⠀⠀⠚⢻⣶⡒⡖⠒⠒⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠂⠀⠀⠀⠶⣶⣒⠒⠒⎥
⎢⢽⢼⠅⠀⠀⠇⠀⢠⠀⠀⠀⠀⢸⠬⠿⣧⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠭⠓⢯⣄⡀⎥
⎢⣍⣿⡅⠀⠀⠀⠀⢸⠀⠀⠀⣀⢸⠀⠀⠉⡻⣮⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⡀⠀⠀⠘⠺⢭⎥
⎢⠆⠶⣷⠀⠀⠀⠀⠀⠀⠂⠀⠶⠀⠀⠀⠀⠀⠘⢻⣶⡖⠒⠂⠀⠀⠀⠀⠀⠀⠀⠀⠒⠒⠾⣿⡇⠀⠀⠀⠀⎥
⎢⠅⠅⣿⠀⠀⠀⠀⠀⠨⠅⠀⠀⠀⠀⠀⠀⠀⠀⢸⠉⠿⣧⡄⠀⠀⠀⠀⠀⠀⠀⠀⣽⡶⠄⠀⠁⠀⠀⠀⠀⎥
⎢⣿⠀⠙⣶⣦⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠉⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠉⠁⠀⠀⠀⠀⠀⠀⎥
⎢⢻⡆⠀⠘⠛⢳⣰⣴⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣾⡇⠀⢰⠀⠀⠛⠻⣶⣆⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢿⣧⠀⠘⠂⠀⢰⠀⠙⠛⢦⣴⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠼⠿⣤⠀⠀⠀⠸⠀⠠⠄⠈⠿⠳⠀⠀⠀⠀⠀⢠⠀⣄⣤⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠄⠤⣿⠀⠀⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠀⠀⢠⣸⡄⠘⠏⠇⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⎥
⎢⣄⢠⡿⠀⠀⡀⠀⠀⠀⠀⠀⠀⢠⡄⡄⡄⠀⠈⠿⠿⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⎥
⎢⣿⣸⡄⠀⠀⡇⠀⢀⠀⠀⠀⠀⢸⢻⡽⣄⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⎥
⎣⣸⣿⡀⠀⠀⠀⠀⢸⠀⠀⠀⠀⢸⠀⠀⠹⡞⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⎦

VectorField

43524×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

(, )

In [13]:
q = (Q * K1 * Q') \ (Q * f1);

In [14]:
showDoFResults(q, :uvec, name="displacement");

In [15]:
S = solveStress(q, DoFResults=true);

In [16]:
S1 = Q' * S * Q

syz = showDoFResults(S1, :syz, visible=true)
showDoFResults(S1, :sx)
showDoFResults(S1, :sy)
showDoFResults(S1, :sz)
showDoFResults(S1, :sxy)
showDoFResults(S1, :szx)

plotOnPath("path", syz)

9

In [17]:
R = nodePositionVector(prob)
R = nodesToElements(R, onPhysicalGroup="left")

n2 = normalVector(prob, "left")

S2 = nodesToElements(S, onPhysicalGroup="left")

τ2 = S2 * n2;

In [18]:
showElementResults(τ2, name="shear stress")

10

In [19]:
M2 = R × (S2 * n2);

In [20]:
ez = VectorField(prob, [field("left", fx=0, fy=0, fz=1)]);

In [21]:
M2z = M2 * ez;

In [22]:
integrate(prob, "left", M2z)

-10512.692393511097

In [23]:
umax = probe(q, 25, 0, 200)[2]

1.6268333202

In [24]:
umax / 25 / 200 * 2e5 / 2 / (1 + 0.3) * 2^3 * (50 * π) / 3

10483.78996380597

In [25]:
n1 = normalVector(prob, "mid");

In [26]:
showDoFResults(n1, name="normal vector")

11

In [27]:
S2 = nodesToElements(S, onPhysicalGroup="mid")
showElementResults(S2 * n1, name="shear stress mid")

12

In [28]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [29]:
gmsh.finalize()